In [1]:
!pip install simpy

In [7]:
import simpy
import random
random.seed(1)

In [10]:
def coffee_machine(env, worker):
  while True:
    print(f'Coffee machine ready to use at {env.now: .2f}.')
    random_inter_arrival_time = random.uniform(60, 180) # We run out of coffee between 1 and 3 hours
    yield env.timeout(random_inter_arrival_time)
    print(f'Coffee over! Call operator at {env.now: .2f}.')
    yield env.process(refilling(env, worker))


def clean_floor(env, worker):
  """Cleaning job takes less priority"""
  while True:
    #Start cleaning an area
    time_to_clean = 120
    while time_to_clean:
      with worker.request(priority=1) as req:
        yield req
        print(f'Cleaning started at {env.now: .2f} and time to clean = {time_to_clean: .2f}')
        try:
          start = env.now
          yield env.timeout(time_to_clean)
          time_to_clean = 0
        except simpy.Interrupt:
          time_to_clean -= env.now - start
          print(f'Cleaning interrupted at {env.now: .2f} and time left to clean = {time_to_clean: .2f}')

def refilling(env, worker):
  """Refilling takes high priority"""
  with worker.request(priority = 0) as req:
      yield req
      refill_time = 15
      yield env.timeout(refill_time) # takes 15 minutes for refilling
      print(f'Refilled. Machine ready to use at {env.now: .2f}.')

#Create an evironment and start the processes
env = simpy.Environment()
worker = simpy.PreemptiveResource(env, capacity = 1)
env.process(coffee_machine(env, worker))
env.process(clean_floor(env, worker))

env.run(until = 24*60)


Coffee machine ready to use at  0.00.
Cleaning started at  0.00 and time to clean =  120.00
Coffee over! Call operator at  100.00.
Cleaning interrupted at  100.00 and time left to clean =  20.00
Refilled. Machine ready to use at  115.00.
Coffee machine ready to use at  115.00.
Cleaning started at  115.00 and time to clean =  20.00
Cleaning started at  135.00 and time to clean =  120.00
Coffee over! Call operator at  215.00.
Cleaning interrupted at  215.00 and time left to clean =  40.00
Refilled. Machine ready to use at  230.00.
Coffee machine ready to use at  230.00.
Cleaning started at  230.00 and time to clean =  40.00
Cleaning started at  270.00 and time to clean =  120.00
Coffee over! Call operator at  330.00.
Cleaning interrupted at  330.00 and time left to clean =  60.00
Refilled. Machine ready to use at  345.00.
Coffee machine ready to use at  345.00.
Cleaning started at  345.00 and time to clean =  60.00
Cleaning started at  405.00 and time to clean =  120.00
Coffee over! Call

In [50]:
def coffee_machine(env):
  while True:
    print(f'Coffee machine ready to use at {env.now: .2f}.')
    random_inter_arrival_time = random.uniform(60, 180) # We run out of coffee between 1 and 3 hours
    yield env.timeout(random_inter_arrival_time)
    print(f'Coffee over! Call operator at {env.now: .2f}.')
    cleaning.interrupt()
    global refilled_event
    yield refilled_event
    refilled_event = env.event()



def clean_floor(env, worker):
  """Cleaning job takes less priority"""
  time_to_clean = 120
  while True:
    try:
      while time_to_clean:
        print(f'Cleaning started at {env.now: .2f} and time to clean = {time_to_clean: .2f}')
        with worker.request() as req:
          yield req
          start = env.now
          yield env.timeout(time_to_clean)
          time_to_clean = 120
    except simpy.Interrupt:
      time_to_clean -= env.now - start
      print(f'Cleaning interrupted at {env.now: .2f} and time left to clean = {time_to_clean: .2f}')
      if time_to_clean == 0:
        time_to_clean = 120
      yield env.process(refilling(env, worker))

def refilling(env, worker):
  """Refilling takes high priority"""
  with worker.request() as req:
    refill_time = 15
    yield req
    yield env.timeout(refill_time) # takes 15 minutes for refilling
    print(f'Refilled. Machine ready to use at {env.now: .2f}.')
    refilled_event.succeed()

#Create an evironment and start the processes
env = simpy.Environment()
refilled_event = env.event()
worker = simpy.Resource(env, capacity = 1)
env.process(coffee_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until = 24*60)


Coffee machine ready to use at  0.00.
Cleaning started at  0.00 and time to clean =  120.00
Coffee over! Call operator at  86.25.
Cleaning interrupted at  86.25 and time left to clean =  33.75
Refilled. Machine ready to use at  101.25.
Coffee machine ready to use at  101.25.
Cleaning started at  101.25 and time to clean =  33.75
Cleaning started at  135.00 and time to clean =  120.00
Coffee over! Call operator at  216.41.
Cleaning interrupted at  216.41 and time left to clean =  38.59
Refilled. Machine ready to use at  231.41.
Coffee machine ready to use at  231.41.
Cleaning started at  231.41 and time to clean =  38.59
Cleaning started at  270.00 and time to clean =  120.00
Coffee over! Call operator at  326.18.
Cleaning interrupted at  326.18 and time left to clean =  63.82
Refilled. Machine ready to use at  341.18.
Coffee machine ready to use at  341.18.
Cleaning started at  341.18 and time to clean =  63.82
Coffee over! Call operator at  403.76.
Cleaning interrupted at  403.76 and 